# The Set Cover Problem DQM
CDL Quantum Hackathon 2021

In [1]:
# Ziwei Qiu, ziweiqiu@g.harvard.edu
import os
os.chdir('..')
from dimod import BinaryQuadraticModel
from dimod import ExactSolver
from neal import SimulatedAnnealingSampler
from itertools import combinations
from dwave.system import LeapHybridSampler
import numpy as np
import pandas as pd
from utils.data import read_inventory_optimization_data

In [2]:
def build_setcover_bqm(U, V, verbose = False):
    """Construct BQM for the set cover problem
    Args:
        U (array-like):
            A set of elements defining the universe
        V (array of sets):
            Array of subsets
    Returns:
        Binary quadratic model instance
        x: variable
    """
    
    # Create indicator variables
    I = []
    for i in range(len(V)):
        I.append([1 if U[a] in V[i] else 0 for a in range(len(U))])
    
    if verbose:
        print('Indicator variables: I_i,a',I)
    
    # Lagrange multipliers A>B>0
    A = 2
    B = 1
    
    ##@  Binary Quadratic Model @##
    bqm = BinaryQuadraticModel('BINARY')

    # Add linear terms
    # x linear terms
    x = [bqm.add_variable('x_'+str(i+1), A*sum(I[i])+B) for i in range(0,len(V))]
    if verbose:
        print('x variables:',x)

    # y_am linear terms
    y = []
    for a in range(1,len(U)+1):
        y.append([bqm.add_variable('y_('+str(a)+', '+str(m)+')', A*(m**2-1)) for m in range(1,len(V)+1)])
    if verbose:
        print('y variables:',y)

    # Add quadratic terms

    # x_i-x_j terms
    for i in range(1,len(V)+1):
        for j in range(i+1,len(V)+1):
            key = ('x_' + str(i), 'x_' + str(j))
            bqm.quadratic[key] = 2*A*np.dot(np.array(I[i-1]),np.array(I[j-1]))

    # y_am - y_an terms
    for m in range(1,len(V)+1):
        for n in range(m+1,len(V)+1):
            for a in range(1,len(U)+1):
                key = ('y_('+str(a)+', '+str(m)+')', 'y_('+str(a)+', '+str(n)+')')
                bqm.quadratic[key] = 2*A*(1+m*n)

    # x_i-y_am terms
    for i in range(1,len(V)+1):
        for m in range(1,len(V)+1):
            for a in range(1,len(U)+1):
                key = ('x_' + str(i), 'y_('+str(a)+', '+str(m)+')')
                bqm.quadratic[key] = -2*A*m*I[i-1][a-1]
    return bqm, x

def solve_bqm(bqm, x, sampler):
    response = sampler.sample(bqm)
    best_solution = response.first.sample
    best_solution = [best_solution[i] for i in x]
    print(best_solution)
    
    return best_solution

# Implementation

In [3]:
# Define a simple set cover problem
U = list(set(np.random.randint(10, size=(10))))

V = [set(U[i] for i in np.random.randint(len(U), size=(8))) for j in range(5)]

print('The universe is',U)
print('Number of elements in the universe: {:d}'.format(len(U)))

print('There are {:d} collections:'.format(len(V)),V)
print('Number of sets: N={:d}'.format(len(V)))

The universe is [1, 2, 4, 5, 8, 9]
Number of elements in the universe: 6
There are 5 collections: [{1, 2, 4, 8, 9}, {1, 4, 5, 8, 9}, {1, 2, 4, 5}, {1, 2, 4}, {1, 2, 4, 5, 9}]
Number of sets: N=5


### Solve the Set Cover Problem with Simulated Annealing

In [4]:
bqm,x = build_setcover_bqm(U, V)
best_solution = solve_bqm(bqm, x, SimulatedAnnealingSampler())

[0, 0, 1, 0, 1]


### Solve the Set Cover Problem with Quantum Annealing (Leap Hybrid Solver)

In [5]:
bqm,x = build_setcover_bqm(U, V)
best_solution = solve_bqm(bqm, x, LeapHybridSampler())

[0, 1, 1, 0, 0]


# Grocery Data 
## Small dataset

In [6]:
inventory, supplier_inventory = read_inventory_optimization_data(os.path.join(os.getcwd(),'data/small-cost-mock.csv'))

In [7]:
print('There are {:d} items in the universe.\n'.format(len(inventory)))
print('There are {:d} suppliers.\n'.format(len(supplier_inventory)))

# Build the BQM
bqm,x = build_setcover_bqm(inventory, supplier_inventory)

# Quantum Annealing
print('Solution:')
best_solution = solve_bqm(bqm, x, LeapHybridSampler())
print('There are {:d} suppliers selected.'.format(sum(best_solution)))
suppliers = [f'supplier{i}' for i in np.where(best_solution)[0]]
print('Selected Suppliers:', suppliers)

There are 20 items in the universe.

There are 10 suppliers.

Solution:
[0, 1, 1, 0, 0, 0, 1, 0, 1, 0]
There are 4 suppliers selected.
Selected Suppliers: ['supplier1', 'supplier2', 'supplier6', 'supplier8']


## Medium dataset

In [8]:
inventory, supplier_inventory = read_inventory_optimization_data(os.path.join(os.getcwd(),'data/medium-cost-mock.csv'))

In [9]:
print('There are {:d} items in the universe.\n'.format(len(inventory)))
print('There are {:d} suppliers.\n'.format(len(supplier_inventory)))

# Build the BQM
bqm,x = build_setcover_bqm(inventory, supplier_inventory)

# Quantum Annealing
print('Solution:')
best_solution = solve_bqm(bqm, x, LeapHybridSampler())
print('There are {:d} suppliers selected.'.format(sum(best_solution)))
suppliers = [f'supplier{i}' for i in np.where(best_solution)[0]]
print('Selected Suppliers:', suppliers)

There are 100 items in the universe.

There are 40 suppliers.

Solution:
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
There are 36 suppliers selected.
Selected Suppliers: ['supplier0', 'supplier1', 'supplier2', 'supplier4', 'supplier5', 'supplier6', 'supplier7', 'supplier8', 'supplier9', 'supplier10', 'supplier11', 'supplier12', 'supplier13', 'supplier14', 'supplier16', 'supplier17', 'supplier18', 'supplier19', 'supplier20', 'supplier22', 'supplier23', 'supplier24', 'supplier25', 'supplier26', 'supplier27', 'supplier28', 'supplier30', 'supplier31', 'supplier32', 'supplier33', 'supplier34', 'supplier35', 'supplier36', 'supplier37', 'supplier38', 'supplier39']


## Large dataset

In [10]:
inventory, supplier_inventory = read_inventory_optimization_data(os.path.join(os.getcwd(),'data/large-cost-mock.csv'))

In [11]:
print('There are {:d} items in the universe.\n'.format(len(inventory)))
print('There are {:d} suppliers.'.format(len(supplier_inventory)))

# Build the BQM
bqm,x = build_setcover_bqm(inventory, supplier_inventory)

# Quantum Annealing
best_solution = solve_bqm(bqm, x, LeapHybridSampler())

suppliers = [f'supplier{i}' for i in np.where(best_solution)[0]]
print('Selected Suppliers:', suppliers)

There are 200 items in the universe.

There are 80 suppliers.
[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Selected Suppliers: ['supplier0', 'supplier2', 'supplier3', 'supplier4', 'supplier5', 'supplier6', 'supplier7', 'supplier8', 'supplier9', 'supplier10', 'supplier11', 'supplier12', 'supplier13', 'supplier14', 'supplier15', 'supplier16', 'supplier17', 'supplier18', 'supplier19', 'supplier20', 'supplier21', 'supplier22', 'supplier23', 'supplier24', 'supplier25', 'supplier26', 'supplier27', 'supplier28', 'supplier29', 'supplier30', 'supplier31', 'supplier32', 'supplier33', 'supplier34', 'supplier35', 'supplier36', 'supplier37', 'supplier38', 'supplier39', 'supplier40', 'supplier41', 'supplier42', 'supplier43', 'supplier44', 'supplier45', 'supplier46', 'supplier47', 'supplier48', 'supplier49',

## Extra Large Dataset

In [13]:
inventory, supplier_inventory = read_inventory_optimization_data(os.path.join(os.getcwd(),'data/extra_large-cost-mock.csv'))

In [14]:
print('There are {:d} items in the universe.\n'.format(len(inventory)))
print('There are {:d} suppliers.'.format(len(supplier_inventory)))

# Build the BQM
bqm,x = build_setcover_bqm(inventory, supplier_inventory)

# Quantum Annealing
best_solution = solve_bqm(bqm, x, LeapHybridSampler())

suppliers = [f'supplier{i}' for i in np.where(best_solution)[0]]
print('Selected Suppliers:', suppliers)

There are 1000 items in the universe.

There are 120 suppliers.


KeyboardInterrupt: 